# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("C:\\Users\\beatr\\Documents\\Beatriz\\Ironhack\\lab-dataframe-calculations\\your-code\\Orders.zip")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Sub Problem 1: How to aggregate the amount_spent for unique customers?:

orders_agg = orders[["amount_spent"]].groupby(orders["CustomerID"]).sum()
orders_agg

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [4]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

stats = orders_agg.describe().transpose()

#stats["IQR"] = stats["75%"] - stats["25%"]
#stats["UpperBound"] = ((stats["75%"]) + (1.5 * stats["IQR"]))
#stats["LowerBound"] = ((stats["25%"]) - (1.5 * stats["IQR"]))

#Add the column with the missing information : 95%
stats["95%"] = np.percentile(orders_agg,95)
stats

,count,mean,std,min,25%,50%,75%,max,95%
amount_spent,4339.0,2053.793018,8988.248381,0.0,307.245,674.45,1661.64,280206.02,5840.182


In [6]:
#Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

orders_agg['Type of Client'] = orders_agg["amount_spent"].map(lambda x: "VIP" if x > stats.iloc[0]["95%"] else "Preferred" if x > stats.iloc[0]["75%"] else "Not VIP nor Preferred")
orders_agg

,amount_spent,Type of Client
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Not VIP nor Preferred
...,...,...
18280,180.60,Not VIP nor Preferred
18281,80.82,Not VIP nor Preferred
18282,178.05,Not VIP nor Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
orders_agg2 = pd.DataFrame(orders.groupby(["CustomerID", "Country"])["amount_spent"].sum())
orders_agg2
orders_agg2.reset_index(inplace = True)
orders_agg2

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [8]:
inner_join = pd.merge(orders_agg2, orders_agg ,on ='CustomerID', how ='inner')
inner_join

,CustomerID,Country,amount_spent_x,amount_spent_y,Type of Client
0,12346,United Kingdom,77183.60,77183.60,VIP
1,12347,Iceland,4310.00,4310.00,Preferred
2,12348,Finland,1797.24,1797.24,Preferred
3,12349,Italy,1757.55,1757.55,Preferred
4,12350,Norway,334.40,334.40,Not VIP nor Preferred
...,...,...,...,...,...
4342,18280,United Kingdom,180.60,180.60,Not VIP nor Preferred
4343,18281,United Kingdom,80.82,80.82,Not VIP nor Preferred
4344,18282,United Kingdom,178.05,178.05,Not VIP nor Preferred
4345,18283,United Kingdom,2094.88,2094.88,Preferred


In [12]:
results = inner_join.groupby('Country')['Type of Client'].apply(lambda x: (x=='VIP').sum()).reset_index(name='count')
results.sort_values(by = "count", ascending = False)

,Country,count
35,United Kingdom,177
14,Germany,10
13,France,9
32,Switzerland,3
0,Australia,2
26,Portugal,2
19,Japan,2
30,Spain,2
10,EIRE,2
12,Finland,1


In [13]:
results.sort_values(by = "count", ascending = False).head(1)

,Country,count
35,United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [18]:
results2 = inner_join.groupby('Country')['Type of Client'].apply(lambda x: (x != 'Not VIP nor Preferred').sum()).reset_index(name='count')
results2.sort_values(by = "count", ascending = False)

,Country,count
35,United Kingdom,932
14,Germany,39
13,France,29
3,Belgium,12
32,Switzerland,9
30,Spain,9
26,Portugal,7
24,Norway,7
18,Italy,5
12,Finland,5


In [20]:
results2.sort_values(by = "count", ascending = False).head(1)

,Country,count
35,United Kingdom,932
